# Reading the rest of HYDRUS files [Stage 2]
![The project files](../Assets/Project_files_2.png)

## Importing and loading dataset

In [ ]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from NesrHydrusAnalyst import *

In [ ]:
src = '../Datasets/H3D2_SandDitch0011'

## Unreadable files
Encrypted files that are converted to text elswhere
1. `h.out`
2. `Q.out`
3. `v.out`
3. `th.out`
4. `MESHTRIA.000`
4. `DOMAIN.IN`

## Reading previous implemented files
### Reading the _mesh files_
1. `H.TXT`
2. `V.TXT`
3. `TH.TXT`
4. `MESHTRIA.TXT`
### Spread-info files

1. `Cum_Q.out`
1. `h_Mean.out`
1. `v_Mean.out`
5. `Run_Inf.out`
### the _information files_
1. `A_Level.out`
2. `ATMOSPH.IN`
3. `Balance.out`
4. `DIMENSIO.IN`
5. `Run_Inf.out` (Repeated)
6. `SELECTOR.IN`

## Files to be implemented here


### Flux-info files

1. `Boundary.out`
1. `BOUNDARY.IN`
1. `Check.out`